In [ ]:
#hidden cell to be executed BEFORE the presentation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from IPython.lib.display import YouTubeVideo
PP_list = {'Al': 'Al_lda.oe01.recpot'}
import fortecubeview

<center>
    <h1>The (almost) all-Python DFTpy and QEpy softwares</h1>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Pavanello Research Group</h1></p><p><h2>Rutgers University-Newark</h2></p></td>
      <td><img src="figures/logos/run.png" width=200 height=200 /></td>
  </tr>
  <tr>
    <td></td>
    <td> http://prg.rutgers.edu</td>
  </tr>
</table>

#### LLNL Computational Chemistry & Materials Science Summer Institute -- June 13, 2023



# Ready to get back to work?

<div class="alert alert-block alert-success">
    <center><b><a href="https://classroom.github.com/a/Lk6pRwki">Lecture 2 assignment</a></b></center>
    </div>

# Learning Goals

#### Theory and techniques

<div class="alert alert-block alert-success">
    <h4>Understand the theory of DFT</h4>
    <ul>
        <li> Orbital-free DFT</li>
        <li> Kohn-Sham DFT</li>
    </ul>
    <p style="text-align: center;"><small>Work individually or in groups. Time: 30'</small></p>
</div>

<div class="alert alert-block alert-success">
    <h4>GitHub and Google Colaboratory</h4>
    <ul>
        <li> Track your work with Git (GitLab, GitHub)</li>
        <li> Jupyter Notebooks on the Google Colaboratory "colab" </li>
    </ul>
    <p style="text-align: center;"><small>Work individually. Time: 15'</small></p>
</div>




# Learning Goals

####  Coding


<div class="alert alert-block alert-danger">
    <h4>Be able to run simulations... in Python</h4>
    <ul>
        <li> Orbital-free DFT with DFTpy</li>
        <li> Plot results with matplotlib</li>
        <li> Make your own $T_s[n]$! </li>
    </ul>
    <p style="text-align: center;"><small>Work individually or in groups. Time: 30'</small></p>
</div>

<div class="alert alert-block alert-danger">
    <h4>Kohn-Sham DFT</h4>
    <ul>
        <li> Kohn-Sham DFT with QEpy</li>
        <li> Make your own $E_{xc}[n]$! </li>
    </ul>
    <p style="text-align: center;"><small>Work individually or in groups. Time: 45'</small></p>
</div>

# Google colaboratory (colab)

<div class="alert alert-block alert-success">
    <center><b><a href="https://colab.research.google.com/">https://colab.research.google.com/</a></b></center>
    </div>
    
- Free and easy to use Jupyter Notebook
- You can run both DFTpy and QEpy with it
- Hit "new notebook" (lower right)


#### Challenge
- Write a minimal code in a "colab" notebook
- Verify that it runs and produces the desired output
- Reproduce (paste) the code in this notebook 
- Paste here a link to your new colab notebook in a separate markdown cell

##### for example:

In [ ]:
print('Riding around...')
for i in range(5):
    print(str(i+1)+' times,')
print('Happy to ride!')

# Understand the Theory of DFT
#### Challenge

- Write down the Kohn-Sham DFT energy density functional.

$$
E[n]=\ldots
$$

### ...understand the Theory of DFT
- What is the difference between the noninteracting kinetic energy functional, $T_s[n]$, and the total kinetic energy, $T[n]$?


### ...understand the Theory of DFT
- Write down an expression for $T_s[n]$ for a uniform electron gas system in terms of its KS orbitals, $\Psi_i(\mathbf{r})=\frac{1}{\Omega^{1/2}}e^{i\mathbf{k}_i\cdot\mathbf{r}}$.


### ...understand the Theory of DFT
- Be ready to discuss it with others or with the class.

Write here your discussion points

# Git and GitHub

- Did you use Git and/or GitHub before? (show hands)
- ...if not, pair with another student who has worked with Git/GitHub before
- Sign in/up GitHub
- Share your GitHub handle with me. I will add you as "student" to a GitHub Classroom.

<div class="alert alert-block alert-success">
    <center><b><a href="https://classroom.github.com/a/IcT1ep-_">GitHub tutorial</a></b></center>
    </div>

# Let's run simulations on colab

 - Prepare a new, empty notebook on the colab
 - visualize `dftpy.ipynb` from LLNL-2-yourname
 - Cut and paste the content of each cell in the colab notebook
 - run it!

# Understanding the `dftpy.ipynb` notebook

### First a model system of bulk Al (cubic cell) is generated with `ASE`

In [ ]:
from ase.build import bulk
atoms = bulk('Al', 'fcc', a=4.05, cubic=True)

In [ ]:
ions = Ions.from_ase(atoms)

<div class="alert alert-block alert-success">
    <b>Understand more deeply:</b>
    Try a different bulk symmetry (bcc?) or make a surface. Visualize the structures.
    </div>

In [ ]:
from ase.io import write
write('image.png', atoms)
from IPython.display import Image
Image('image.png')

# Can visualize dynamically inside the notebook

In [ ]:
from ase.visualize import view
view(atoms, viewer='ngl')

# Let's try other systems
#### Surfaces?

In [ ]:
from ase.build import fcc111
atoms = fcc111('Al', size=(2,2,3), vacuum=10.0)
view(atoms, viewer='ngl')

#### The FFT grid

In [ ]:
nr = ecut2nr(ecut=35, lattice=ions.cell)

In [ ]:
grid = DirectGrid(lattice=ions.cell, nr=nr)

#### Local pseudopotential (external potential)

You can get the pseudo for today's example here:
!wget http://eqe.rutgers.edu/Al_lda.oe01.recpot

More pseudos for OF-DFT are available at:
https://gitlab.com/shaoxc/ofpp

In [ ]:
PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list)

#### Initial guess for the electron density

In [ ]:
rho_ini = DirectField(grid=grid)
rho_ini[:] = ions.get_ncharges()/ions.cell.volume

#### The $E_H[n]$ and $E_{xc}[n]$ functionals

In [ ]:
HARTREE = Functional(type='HARTREE')
XC = Functional(type='XC',name='LDA')

#### The $T_s[n]$ functional

In [ ]:
KE = Functional(type='KEDF',name='x_TF_y_vW')

#### The total energy, $E[n]$, functional

In [ ]:
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)

### Minimize $E[n]-\mu \left[\int n(\mathbf{r}) d\mathbf{r} -N \right]$

In [ ]:
optimization_options = {'econv' : 1e-3*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')
rho = opt.optimize_rho(guess_rho=rho_ini)

# Visualize the electron density
We use fortecubeview from the Evangelista Lab at Emory University
https://github.com/evangelistalab/fortecubeview

In [ ]:
rho.write('rho.cube',ions=ions)

In [ ]:
sumlevel=0.85
print("Isosurface value: ","{:.2E}".format((1.0-sumlevel)*np.max(np.abs(rho))))

In [ ]:
fortecubeview.plot(cubes=['./rho.cube'],sumlevel=sumlevel);

# Challenge
- make a new notebook on the colab
- build a bulk FCC Al
- determine the total energy for lattice constant from $a=2$&#8491; to $a=6$&#8491; over 10 points.
- plot the energy vs. $a$ 

#### Tip

In [ ]:
for i in range(10):
    aloop=2+(6-2)/9*i
    print(aloop)
    # insert here the DFTpy code!!!

# New goal: KS-DFT in Python
- run Quantum ESPRESSO with the Python interface QEpy
- do it on the colab!
- check out the `qepy.ipynb` notebook 

# Understanding the `qepy.ipynb` notebook

#### QEpy driver: Python driver class for Quantum ESPRESSO
This is the main QEpy class

In [ ]:
from qepy.driver import Driver
from qepy.io import QEInput

#### Make an "input file"

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'prefix': "'Al'",
        'pseudo_dir': "'./'",
        'restart_mode': "'from_scratch'"},
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'"},
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points automatic': ['2 2 2 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

You can get the pseudo for today's example here:
!wget http://pseudopotentials.quantum-espresso.org/upf_files/Al.pbe-nl-kjpaw_psl.1.0.0.UPF

More pseudos are available here:
https://www.quantum-espresso.org/pseudopotentials/

#### Initialize QEpy driver

In [ ]:
driver=Driver(qe_options=qe_options, logfile=True)

#### Run the SCF

In [ ]:
driver.scf()

#### Extract information from driver

In [ ]:
driver.get_scf_error()

#### ...even have access to density and wavefunctions!

In [ ]:
rho=driver.get_density()
nr=driver.get_number_of_grid_points()

#### Let's reshape the density to something like `rho[x,y,z]`

In [ ]:
rho_new=rho.reshape(nr,order='F')

Be mindful, Quantum ESPRESSO is coded in Fortran

#### Let's plot the density along the $z$ axis

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho_new[0,10,:])
plt.title('KS-DFT electron density');

#### Cleanup driver
Needed to reset the memory to be able to create other drivers (if needed)

In [ ]:
driver.stop()

# Big Challenge
- Plot energy vs lattice constant for bulk Al (bcc or fcc) with QEpy
- Use the notebook `qepy.ipynb`
- Port to colab...etc
- Be mindful of the computational scaling of KS-DFT compared to OF-DFT

# Final challenge: Coding new DFT methods
Coding new DFT functionals in DFTpy and QEpy is barrierless

#### DFTpy
- define new `Functional`
- add it to the `TotalFunctional`

#### QEpy
- define new potential as a function of the density
- run SCF in a sequential way rather than in one step with `driver.scf()`

# First: write down correct formulas
####  Dirac's exchange
The energy functional is:
$$E_x[n]=-\frac{3}{4}\left(\frac{3}{\pi}\right)^{1/3}\int n^{4/3}(\mathbf{r}) d \mathbf{r}$$

The potential is:
$$v_x(\mathbf{r})=-\left(\frac{3}{\pi}\right)^{1/3}n^{1/3}(\mathbf{r})$$

# Dirac exchange in DFTpy
We need to code a new `Functional`

In [ ]:
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.functional.functional_output import FunctionalOutput
import numpy as np

class DiracExchange(AbstractFunctional):
    def __init__(self):
        self.type = 'x'
        self.name = 'Dirac Exchange'
    def compute(self, rho,**kwargs):
        v_of_r = -rho**(1.0/3.0)*(3.0/np.pi)**(1.0/3.0) # put any potential you like here!!!
        e=(v_of_r*rho).integral()*(3.0/4.0)             # put corresponding energy here!
        functional=FunctionalOutput(name="Exchange", potential=v_of_r, energy=e)
        return functional

# Let's try our new Dirac exchange
#### Instance the classes

In [ ]:
DIRAC = DiracExchange()

In [ ]:
evaluator = TotalFunctional(KE=KE, X=DIRAC, HARTREE=HARTREE, PSEUDO=PSEUDO)

#### Run the optimization with DFTpy

In [ ]:
optimization_options = {'econv' : 1e-3*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')
rho = opt.optimize_rho(guess_rho=rho_ini)

# Dirac exchange in QEpy
- We need to run the SCF iteratively (cannot use `driver.scf`)
- We need to provide an additional external potential with Dirac's exchange

#### Initialize the QEpy driver - this time with iterative SCF

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile=True)

#### Run the SCF with a `for` loop

In [ ]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('It: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

#### Print the total electronic energy

In [ ]:
driver.get_energy()

In [ ]:
driver.embed.etotal

#### What xc functional have we been using?
- This was automatically selected by Quanrum ESPRESSO from the pseudopotential

In [ ]:
driver.get_xc_functional()

#### Close driver

In [ ]:
driver.stop()

#### Let's first make a function for $E_x[n]$

In [ ]:
import numpy as np
def dirac_x(rho,dV):
    v_of_r = -rho**(1.0/3.0)*(3.0/np.pi)**(1.0/3.0) # put any potential you like here!!!
    e=np.sum((v_of_r*rho))*(3.0/4.0)*dV             # put corresponding energy here!
    return v_of_r*2,e*2
# x 2 is for converting Ha to Ry, units used by Quantum ESPRESSO

#### Adding a new exchange-correlation functional

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile=True)

#### Notice the `set_external_potential` method
- extpot is a `numpy.Array` of same shape as the density

In [ ]:
for i in range(60):
    nr=driver.get_number_of_grid_points()
    nnr=np.prod(nr)
    l=driver.get_ions_lattice()
    V=np.dot(l[0],np.cross(l[1],l[2]))
    dV=V/nnr
    extpot, ex = dirac_x(driver.get_density(),dV)
    driver.set_external_potential(potential=extpot, extene=ex, exttype='100')
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('It: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break


In [ ]:
print('The final exchange energy:')
ex

In [ ]:
driver.get_energy()

#### `set_external_potential` method `exttype` options
<table>
  <tr>
    <th>exttype</th>
    <th>term replaced in $v_s(\mathbf{r})$</th>
  </tr>
  <tr>
    <td>000</td>
    <td>external potential</td>
  </tr>
  <tr>
    <td>001</td>
    <td>local pseudopotential</td>
  </tr>
  <tr>
    <td>010</td>
    <td>hartree potential</td>
  </tr>
  <tr>
    <td>011</td>
    <td>hartree and pseudopotential</td>
  </tr>
  <tr>
    <td>100</td>
    <td>exchange-correlation</td>
  </tr>
  <tr>
    <td>101</td>
    <td>exchange-correlation and pseudopotential</td>
  </tr>
  <tr>
    <td>110</td>
    <td>hartree and exchange-correlation</td>
  </tr>
  <tr>
    <td>111</td>
    <td>pseudo+hartree+xc</td>
  </tr>
</table>    

In [ ]:
driver.stop()

# Now implement it yourself on the colab!!
- make a new notebook using `qepy.ipynb` as template
- consult `LLNL_2022_Lecture_2.ipynb` for details on giving QEpy a custom xc potential
- commit to your repo once done

# We are done! Thank you!

And a big thanks to PRG's awesome Postdocs and Graduate Students, but especially
- Dr. Xuecheng Shao for doing everything, really
- Dr. Kaili Jiang, for the TDDFT modules and modularization of DFTpy
- Andres Cifuentes for installation and documentation support
- Jessica Martinez for documentation support
- Dr Xin Chen
- Valeria Rios Vargas